# Тестирование моделей для предсказания activity

## Импорты, константы, дефолтные настройки

In [1]:
# !uv pip install scikit-learn\<1.6.0

In [2]:
# Импорт модуля для сохранения пайплайна
import os
import joblib

In [3]:
# Импортируем классы для создания пайплайна
from pipeline_classes import *

In [4]:
# Убираем необязательные предупреждения
import warnings
warnings.filterwarnings('ignore')

# Настраиваем полное отображение всех строк таблицы
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

# Скрываем журнал с триалами
optuna.logging.set_verbosity(optuna.logging.WARNING)

## Загрузка данных

In [5]:
start = pd.read_csv('../start_p2.csv')

df_final = pd.read_csv('../final_p2.1.csv')

# every (X, ir, pot2) formula properties is nan or formula is nan or activity is nan
condition = ~((df_final[["X", "IR", "pot2"]].isna().sum(axis=1) == 3) +\
              start[["formula", "activity"]].isna().any(axis=1))

df_final = df_final.drop(columns='id')
df_final = df_final[condition]
df_final['activity'] = df_final['activity'].astype(int)
display(df_final.head(3))
df = df_final

,Km,Vmax,activity,X,IR,pot2,ph,temp,dstr,lgCmin,lgCmax,Cmin,Cmax,lgCconst,Cconst,lgCcat,Ccat,lgvolume,MolWt,PEOE_VSA7,PEOE_VSA9,VSA_EState8,Kappa2,BalabanJ,MinAbsEStateIndex,MinEStateIndex,EState_VSA6,VSA_EState4,PEOE_VSA8,MinPartialCharge,EState_VSA4,SMR_VSA7,Complexity1,TPSA1,TPSA2,TPSA,XLogP,MaxEStateIndex.1,MaxEStateIndex.2,MinPartialCharge.1,MaxPartialCharge.1,BCUT2D_CHGLO
7,NaN,NaN,1,2.215,NaN,-0.140000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,0.02,NaN,1,2.750,1.114286,-0.015429,5.0,55.0,NaN,NaN,NaN,NaN,NaN,1.602060,40.0,-0.30103,0.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.5,52.0,NaN,NaN,6.0,5.992739,-0.254557,-0.254557,NaN
14,0.01,0.001,1,2.675,NaN,-0.130000,3.5,50.0,3.0,-0.69897,1.69897,0.2,50.0,-0.221849,0.6,-1.69897,0.02,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.5,52.0,NaN,NaN,6.0,5.992739,-0.254557,-0.254557,NaN


In [6]:
# Загружаем список входных признаков, коррелирующих с целевым
with open("corr_with_activity.txt", "r") as file_activity:
    corr_with_activity = file_activity.read().splitlines()

print(f'Количество коррелирующих признаков: {len(corr_with_activity)}')
print(corr_with_activity)

Количество коррелирующих признаков: 23
['MolWt', 'X', 'lgvolume', 'temp', 'EState_VSA4', 'lgCconst', 'PEOE_VSA7', 'MaxPartialCharge.1', 'PEOE_VSA8', 'TPSA1', 'lgCcat', 'MinPartialCharge.1', 'BalabanJ', 'VSA_EState8', 'MinEStateIndex', 'MinAbsEStateIndex', 'ph', 'pot2', 'Complexity1', 'MaxEStateIndex.1', 'MaxEStateIndex.2', 'TPSA2', 'IR']


## Предобработка данных

In [7]:
print(f'Размер первоначального датафрейма: {df.shape}')

Размер первоначального датафрейма: (3178, 42)


In [8]:
# Удаляем дубликаты
df = df.drop_duplicates()
print(f'Размер датафрейма после удаления дубликатов: {df.shape}')

Размер датафрейма после удаления дубликатов: (2959, 42)


In [9]:
df['activity'].value_counts()

activity
1     2578
2      129
3      110
5       51
6       23
9       12
7       10
10       7
15       6
4        6
14       6
12       5
17       4
19       4
11       2
20       2
18       1
31       1
13       1
28       1
Name: count, dtype: int64

In [10]:
# Удаляем элемент, у которого значение целевого призкака равно 4 (всего один элемент)
df = df[df['activity'].astype(int).isin([1, 2, 3, 5, 6, 9, 7])]
print(f'Размер датафрейма после удаления элементов с activity = 4: {df.shape}')

Размер датафрейма после удаления элементов с activity = 4: (2913, 42)


In [13]:
# Преобразование типов и замена "no" на NaN
df['Vmax'] = pd.to_numeric(df['Vmax'].replace("no", np.nan), errors='coerce')
df['Km'] = pd.to_numeric(df['Km'].replace("no", np.nan), errors='coerce')

invalid_values = list(df_final['ph'][~df_final['ph'].apply(lambda x: pd.to_numeric(x, errors='coerce')).notna()])
df['ph'] = pd.to_numeric(df['ph'].where(lambda ph: ~ph.isin(invalid_values)), errors='coerce')

# Проверяем что сделали корректно
print(df[['Vmax', 'Km', 'ph']].dtypes)

Vmax    float64
Km      float64
ph      float64
dtype: object


In [14]:
# Оставляем в датафрейме только те признаки, которые коррелируют с целевым
df = df[corr_with_activity + ['activity']]
print(f'Размеры датафрейма: {df.shape}')
print(f'Количество дубликатов после удаления признаков: {df.duplicated().sum()}')

Размеры датафрейма: (2913, 24)
Количество дубликатов после удаления признаков: 277


## Тестирование моделей

### Разбивка датафрейма на тренировочную и тестовыю выбрки

In [15]:
# Разделяем на целевой признак и на входные
X = df.drop('activity', axis=1)
y = pd.DataFrame(df['activity'])

# Разделяем данные на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y)

# Стераем названия колонок
X.columns.name = ''
X_train.columns.name = ''
X_test.columns.name = ''

# Преобразуем целевой признак в формат даафрейма
y = pd.DataFrame(y)
y_train = pd.DataFrame(y_train)
y_test = pd.DataFrame(y_test)

# Стераем старые индексы
X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

# Вывод размеров датафреймов
print(f"Размер X_train: {X_train.shape}")
print(f"Размер y_train: {y_train.shape}")
print(f"Размер X_test: {X_test.shape}")
print(f"Размер y_test: {y_test.shape}")

Размер X_train: (2184, 23)
Размер y_train: (2184, 1)
Размер X_test: (729, 23)
Размер y_test: (729, 1)


## Сохранение тестовой выборки для интерпретации модели

In [16]:
# Создаем папку
os.makedirs('test_train_data', exist_ok=True)

# Сохраняем данные
X_train.to_csv('test_train_data/X_train.csv', index=False)
y_train.to_csv('test_train_data/y_train.csv', index=False)
X_test.to_csv('test_train_data/X_test.csv', index=False)
y_test.to_csv('test_train_data/y_test.csv', index=False)

# Проверка, что файлы сохранены
files = ['test_train_data/X_train.csv', 'test_train_data/y_train.csv',
         'test_train_data/X_test.csv', 'test_train_data/y_test.csv']
if all(os.path.exists(file) for file in files):
    print("Данные успешно сохранены.")
else:
    print("Ошибка: Не все данные сохранились!")

Данные успешно сохранены.


### Подбор параметров, сравнение моделей

In [17]:
# Создаём экземпляр класса, задаём необходимые данные и настройки, запускаем подбор
study = Study(X_train, y_train, X_test, y_test, cv_num=5)
models_for_test_list = ['LogisticRegression', 'KNNClassifier', 'SVC', 'DecisionTreeClassifier', 'RandomForestClassifier', 'CatBoostClassifier', 'MLPClassifier']
study.set_models_list(models_for_test_list)
study.set_model_score('roc_auc')
study.set_trials_num(20)
study.set_show_trials_num(1)
study.run()

  0%|          | 0/20 [00:00<?, ?it/s]

LogisticRegression,roc_auc,duration,num_encoding,C,imputer_neighbors,n_quantiles,output_distribution,solver_and_penalty,weights
1,0.880842,00:00:04.384822,QuantileTransformer,48.744425,35,332.0,normal,saga + l2,uniform


  0%|          | 0/20 [00:00<?, ?it/s]

KNNClassifier,roc_auc,duration,num_encoding,algorithm,imputer_neighbors,n_neighbors,n_quantiles,output_distribution,p,weights
1,0.899978,00:00:03.329263,RobustScaler,brute,46,26,NaN,NaN,2,uniform


  0%|          | 0/20 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

RandomForestClassifier,roc_auc,duration,num_encoding,bootstrap,criterion,imputer_neighbors,max_depth,max_features,max_leaf_nodes,min_samples_leaf,min_samples_split,n_estimators,n_quantiles,output_distribution,weights
1,0.947055,00:00:32.641645,PowerTransformer standardize=False,True,entropy,50,13,9,10,4,4,625,NaN,NaN,uniform


  0%|          | 0/20 [00:00<?, ?it/s]

[W 2025-05-01 15:33:36,715] Trial 1 failed with parameters: {'imputer_neighbors': 48, 'weights': 'uniform', 'num_encoding': 'PowerTransformer standardize=True', 'learning_rate': 0.025892799751190677, 'depth': 12, 'l2_leaf_reg': 0.011491094710675756, 'random_strength': 0.1842786974189522, 'bagging_temperature': 0.4430333599023766, 'auto_class_weights': 'Balanced', 'border_count': 213, 'grow_policy': 'SymmetricTree', 'leaf_estimation_iterations': 10, 'leaf_estimation_method': 'Gradient'} because of the following error: KeyboardInterrupt('').
Traceback (most recent call last):
  File "/home/oleg/Programs/.venv/lib/python3.12/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/home/oleg/Documents/science/prediction_clean/models/pipeline_classes.py", line 173, in __call__
    self._pipe_final.named_steps['model'].fit(train_pool,
  File "/home/oleg/Programs/.venv/lib/python3.12/site-packages/catboost/c

KeyboardInterrupt: 

## Стекинг моделей

In [14]:
# Сохраняем наиболее интересные модели в переменные
svc_pipe = study.best_piplines_dict['SVC']
rf_pipe = study.best_piplines_dict['RandomForestClassifier']
cb_pipe = study.best_piplines_dict['CatBoostClassifier']

In [15]:
# Создание ансамбля
ensemble_model = StackingClassifier(
    estimators=[
        ('svc', svc_pipe),
        ('rf', rf_pipe),
        ('cb', cb_pipe)
    ],
    final_estimator=LogisticRegression(),
    cv=5
)

# Обучение ансамбля
ensemble_model.fit(X_train, y_train)

StackingClassifier(cv=5,
                   estimators=[('svc',
                                Pipeline(steps=[('feature_engineering',
                                                 FunctionTransformer(func=<bound method Objective.do_feature_engineering of <pipeline_classes.ObjectiveClassifier object at 0x000001D7D36A2590>>,
                                                                     kw_args={})),
                                                ('imputer', KNNImputer()),
                                                ('num', PowerTransformer()),
                                                ('model',
                                                 SVC(C=34.945108668162575,
                                                     probability=True,
                                                     random_state=38...
                                                 FunctionTransformer(func=<bound method Objective.do_feature_engineering of <pipeline_classes.ObjectiveClassifier object at 0x000001D7D36A2590>>,
                                                                     kw_args={})),
                                                ('imputer',
                                                 KNNImputer(n_neighbors=9,
                                                            weights='distance')),
                                                ('num',
                                                 QuantileTransformer(n_quantiles=478)),
                                                ('model',
                                                 <catboost.core.CatBoostClassifier object at 0x000001D7D4F49D50>)]))],
                   final_estimator=LogisticRegression())

In [16]:
# Получаем предсказания лучшей модели
proba = ensemble_model.predict_proba(X_test)
pred = ensemble_model.predict(X_test)

# Считаем метрики
auroc = roc_auc_score(y_test, proba, multi_class='ovr', average='macro')
accuracy = accuracy_score(y_test, pred)

# Выводим метрики на экран
print(f"AUROC для стекинговой модели: {auroc}")
print(f"Accuracy для стекинговой модели: {accuracy}")

AUROC для стекинговой модели: 1.0
Accuracy для стекинговой модели: 0.9893617021276596


## Сохранение лучших моделей в файлы

In [18]:
# Создаем папку models, если она не существует
os.makedirs('saved_models', exist_ok=True)

# Сохраняем модели в папку models
joblib.dump(svc_pipe, 'saved_models/svc.pkl')
joblib.dump(rf_pipe, 'saved_models/random_forest.pkl')
joblib.dump(cb_pipe, 'saved_models/catboost.pkl')
joblib.dump(ensemble_model, 'saved_models/ensemble_model.pkl')

# Проверка, что файлы существуют
files = ['saved_models/svc.pkl', 'saved_models/random_forest.pkl', 'saved_models/catboost.pkl', 'saved_models/ensemble_model.pkl']
for file in files:
    if os.path.exists(file):
        print(f"Файл {file} успешно создан.")
    else:
        print(f"Ошибка: файл {file} не был создан.")

Файл saved_models/svc.pkl успешно создан.
Файл saved_models/random_forest.pkl успешно создан.
Файл saved_models/catboost.pkl успешно создан.
Файл saved_models/ensemble_model.pkl успешно создан.
